In [2]:

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Concatenate

# Definisikan dimensi input
sequence_length = 10
num_features = 5
static_dim = 3
output_dim = 1

# Input untuk data timeseries
timeseries_input = Input(shape=(sequence_length, num_features), name="timeseries_input")

# GRU Layer
gru_output = GRU(128)(timeseries_input)

# Input untuk data static
static_input = Input(shape=(static_dim,), name="static_input")

# Concatenate GRU output dengan data static
concatenated = Concatenate()([gru_output, static_input])

# Fully connected layer
fc1 = Dense(64, activation="relu")(concatenated)
fc2 = Dense(32, activation="relu")(fc1)
output = Dense(output_dim, activation="linear")(fc2)

# Definisikan model
model = Model(inputs=[timeseries_input, static_input], outputs=output)

# Compile model
model.compile(optimizer="adam", loss="mse")

# Ringkasan model
model.summary()

# Generate synthetic data untuk pelatihan
import numpy as np

def generate_data(num_samples, sequence_length, num_features, static_dim):
    timeseries_data = np.random.rand(num_samples, sequence_length, num_features)
    static_data = np.random.rand(num_samples, static_dim)
    labels = np.random.rand(num_samples, output_dim)
    return timeseries_data, static_data, labels

# Generate data
num_samples = 1000
timeseries_data, static_data, labels = generate_data(num_samples, sequence_length, num_features, static_dim)

# Split data into training and validation sets
train_size = int(0.8 * num_samples)
x_train = [timeseries_data[:train_size], static_data[:train_size]]
y_train = labels[:train_size]
x_val = [timeseries_data[train_size:], static_data[train_size:]]
y_val = labels[train_size:]

# Training model
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluasi model
loss = model.evaluate(x_val, y_val)
print(f"Validation Loss: {loss:.4f}")


2024-06-23 23:59:36.798996: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 23:59:36.799022: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 23:59:36.799801: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-23 23:59:36.803795: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-23 23:59:37.346381: W tensorflow/compiler/tf2

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 timeseries_input (InputLay  [(None, 10, 5)]              0         []                            
 er)                                                                                              
                                                                                                  
 gru (GRU)                   (None, 128)                  51840     ['timeseries_input[0][0]']    
                                                                                                  
 static_input (InputLayer)   [(None, 3)]                  0         []                            
                                                                                                  
 concatenate (Concatenate)   (None, 131)                  0         ['gru[0][0]',             

2024-06-23 23:59:37.898765: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-06-23 23:59:37.898788: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: w4090
2024-06-23 23:59:37.898793: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: w4090
2024-06-23 23:59:37.898816: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 550.90.7
2024-06-23 23:59:37.898832: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 550.78.0
2024-06-23 23:59:37.898836: E external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:244] kernel version 550.78.0 does not match DSO version 550.90.7 -- cannot find working devices in this configuration


25/25 [==============================] - 1s 12ms/step - loss: 0.1121 - val_loss: 0.0890
Epoch 2/10
25/25 [==============================] - 0s 4ms/step - loss: 0.0833 - val_loss: 0.0888
Epoch 3/10
25/25 [==============================] - 0s 4ms/step - loss: 0.0829 - val_loss: 0.0877
Epoch 4/10
25/25 [==============================] - 0s 4ms/step - loss: 0.0812 - val_loss: 0.0881
Epoch 5/10
25/25 [==============================] - 0s 4ms/step - loss: 0.0812 - val_loss: 0.0888
Epoch 6/10
25/25 [==============================] - 0s 4ms/step - loss: 0.0810 - val_loss: 0.0888
Epoch 7/10
25/25 [==============================] - 0s 4ms/step - loss: 0.0840 - val_loss: 0.0892
Epoch 8/10
25/25 [==============================] - 0s 4ms/step - loss: 0.0814 - val_loss: 0.0891
Epoch 9/10
25/25 [==============================] - 0s 4ms/step - loss: 0.0794 - val_loss: 0.0892
Epoch 10/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0892
Validation Loss: 0.0892


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Definisikan model
class TimeseriesStaticModel(nn.Module):
    def __init__(self, sequence_length, num_features, static_dim, output_dim, gru_hidden_dim=128, fc1_dim=64, fc2_dim=32):
        super(TimeseriesStaticModel, self).__init__()
        self.gru = nn.GRU(input_size=num_features, hidden_size=gru_hidden_dim, batch_first=True)
        self.fc1 = nn.Linear(gru_hidden_dim + static_dim, fc1_dim)
        self.fc2 = nn.Linear(fc1_dim, fc2_dim)
        self.fc3 = nn.Linear(fc2_dim, output_dim)

    def forward(self, timeseries_data, static_data):
        gru_output, _ = self.gru(timeseries_data)
        gru_output_last = gru_output[:, -1, :]  # Ambil output terakhir dari GRU
        concatenated = torch.cat((gru_output_last, static_data), dim=1)
        x = torch.relu(self.fc1(concatenated))
        x = torch.relu(self.fc2(x))
        output = self.fc3(x)
        return output

# Generate synthetic data untuk pelatihan
def generate_data(num_samples, sequence_length, num_features, static_dim):
    timeseries_data = np.random.rand(num_samples, sequence_length, num_features)
    static_data = np.random.rand(num_samples, static_dim)
    labels = np.random.rand(num_samples, output_dim)
    return timeseries_data, static_data, labels

# Definisikan dimensi input
sequence_length = 10
num_features = 5
static_dim = 3
output_dim = 1

# Generate data
num_samples = 1000
timeseries_data, static_data, labels = generate_data(num_samples, sequence_length, num_features, static_dim)

# Split data into training and validation sets
train_size = int(0.8 * num_samples)
x_train_timeseries = torch.tensor(timeseries_data[:train_size], dtype=torch.float32)
x_train_static = torch.tensor(static_data[:train_size], dtype=torch.float32)
y_train = torch.tensor(labels[:train_size], dtype=torch.float32)
x_val_timeseries = torch.tensor(timeseries_data[train_size:], dtype=torch.float32)
x_val_static = torch.tensor(static_data[train_size:], dtype=torch.float32)
y_val = torch.tensor(labels[train_size:], dtype=torch.float32)

# Initialize model, loss function, and optimizer
model = TimeseriesStaticModel(sequence_length, num_features, static_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
batch_size = 32

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for i in range(0, train_size, batch_size):
        batch_timeseries = x_train_timeseries[i:i+batch_size]
        batch_static = x_train_static[i:i+batch_size]
        batch_labels = y_train[i:i+batch_size]

        optimizer.zero_grad()
        predictions = model(batch_timeseries, batch_static)
        loss = criterion(predictions, batch_labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/((train_size+batch_size-1)//batch_size):.4f}')

# Evaluasi model
model.eval()
with torch.no_grad():
    val_predictions = model(x_val_timeseries, x_val_static)
    val_loss = criterion(val_predictions, y_val)
    print(f'Validation Loss: {val_loss.item():.4f}')


Epoch [1/10], Loss: 0.1184
Epoch [2/10], Loss: 0.0857
Epoch [3/10], Loss: 0.0835
Epoch [4/10], Loss: 0.0834
Epoch [5/10], Loss: 0.0834
Epoch [6/10], Loss: 0.0833
Epoch [7/10], Loss: 0.0832
Epoch [8/10], Loss: 0.0832
Epoch [9/10], Loss: 0.0831
Epoch [10/10], Loss: 0.0830
Validation Loss: 0.0821


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Concatenate, RepeatVector, TimeDistributed

# Definisikan dimensi input
encoder_sequence_length = 10
decoder_sequence_length = 5
num_features = 5
static_dim = 3
output_dim = 1

# Encoder
encoder_input = Input(shape=(encoder_sequence_length, num_features), name="encoder_input")
encoder_gru = GRU(128, return_state=True, name="encoder_gru")
encoder_output, encoder_state = encoder_gru(encoder_input)

# Decoder
decoder_input = Input(shape=(decoder_sequence_length, num_features), name="decoder_input")
decoder_gru = GRU(128, return_sequences=True, return_state=True, name="decoder_gru")
decoder_output, _ = decoder_gru(decoder_input, initial_state=encoder_state)

# TimeDistributed Dense layer for output sequence
time_distributed = TimeDistributed(Dense(num_features))(decoder_output)

# Model for seq2seq
seq2seq_model = Model(inputs=[encoder_input, decoder_input], outputs=time_distributed)

# Compile seq2seq model
seq2seq_model.compile(optimizer="adam", loss="mse")

# Summary of seq2seq model
seq2seq_model.summary()

# Generate synthetic data untuk pelatihan seq2seq
import numpy as np

def generate_seq2seq_data(num_samples, encoder_sequence_length, decoder_sequence_length, num_features, static_dim):
    encoder_input_data = np.random.rand(num_samples, encoder_sequence_length, num_features)
    decoder_input_data = np.random.rand(num_samples, decoder_sequence_length, num_features)
    decoder_output_data = np.random.rand(num_samples, decoder_sequence_length, num_features)
    static_data = np.random.rand(num_samples, static_dim)
    return encoder_input_data, decoder_input_data, decoder_output_data, static_data

# Generate data
num_samples = 1000
encoder_input_data, decoder_input_data, decoder_output_data, static_data = generate_seq2seq_data(num_samples, encoder_sequence_length, decoder_sequence_length, num_features, static_dim)

# Split data into training and validation sets
train_size = int(0.8 * num_samples)
x_train_encoder = encoder_input_data[:train_size]
x_train_decoder = decoder_input_data[:train_size]
y_train_decoder = decoder_output_data[:train_size]
x_train_static = static_data[:train_size]
x_val_encoder = encoder_input_data[train_size:]
x_val_decoder = decoder_input_data[train_size:]
y_val_decoder = decoder_output_data[train_size:]
x_val_static = static_data[train_size:]

# Train seq2seq model
history = seq2seq_model.fit([x_train_encoder, x_train_decoder], y_train_decoder, epochs=10, batch_size=32, validation_data=([x_val_encoder, x_val_decoder], y_val_decoder))

# Encoder Model
encoder_model = Model(inputs=encoder_input, outputs=encoder_state)

# Decoder Model
decoder_state_input = Input(shape=(128,))
decoder_output, decoder_state = decoder_gru(decoder_input, initial_state=decoder_state_input)
decoder_model = Model(inputs=[decoder_input, decoder_state_input], outputs=[decoder_output, decoder_state])

# Generate output from seq2seq model
def predict_sequence(encoder_input, decoder_input):
    encoder_state = encoder_model.predict(encoder_input)
    decoder_output, _ = decoder_model.predict([decoder_input, encoder_state])
    return decoder_output

# Predict output sequence
predicted_sequence = predict_sequence(x_val_encoder, x_val_decoder)

# Concatenate predicted sequence with static data
predicted_sequence_last = predicted_sequence[:, -1, :]
concatenated = np.concatenate((predicted_sequence_last, x_val_static), axis=1)

# Fully connected layer
fc_input = Input(shape=(predicted_sequence_last.shape[1] + static_dim,))
fc1 = Dense(64, activation="relu")(fc_input)
fc2 = Dense(32, activation="relu")(fc1)
fc_output = Dense(output_dim, activation="linear")(fc2)

# Define final model
final_model = Model(inputs=fc_input, outputs=fc_output)

# Compile final model
final_model.compile(optimizer="adam", loss="mse")

# Summary of final model
final_model.summary()

# Generate synthetic data untuk pelatihan final model
y_final = np.random.rand(num_samples, output_dim)
y_train_final = y_final[:train_size]
y_val_final = y_final[train_size:]

# Train final model
history = final_model.fit(concatenated, y_val_final, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate final model
loss = final_model.evaluate(concatenated, y_val_final)
print(f"Validation Loss: {loss:.4f}")


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_input (InputLayer)  [(None, 10, 5)]              0         []                            
                                                                                                  
 decoder_input (InputLayer)  [(None, 5, 5)]               0         []                            
                                                                                                  
 encoder_gru (GRU)           [(None, 128),                51840     ['encoder_input[0][0]']       
                              (None, 128)]                                                        
                                                                                                  
 decoder_gru (GRU)           [(None, 5, 128),             51840     ['decoder_input[0][0]', 

RuntimeError: Expected hidden size (1, 5, 128), got [1, 10, 128]